In [24]:
import random
import numpy as np
import pandas as pd

from mido import MidiFile
from glob import glob

In [25]:
import shutil, os

def create_only_folders(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir,f))]



shutil.copytree('/Users/cooky/Desktop/groove', './bar_dataset', ignore=create_only_folders)

'./bar_dataset'

In [26]:
midi_list = glob('/Users/cooky/Desktop/groove/**/**/*.mid')

In [27]:
def kit_classify(note):
    cat = ''
    if note in [36]:
        cat = 0
    elif note in [38, 40, 37]:
        cat = 1
    elif note in [48, 50]:
        cat = 2
    elif note in [45, 47]:
        cat = 3
    elif note in [43, 58]:
        cat = 4
    elif note in [46, 26]:
        cat = 5
    elif note in [42, 22, 44]:
        cat = 6
    elif note in [49, 55, 57, 52]:
        cat = 7
    elif note in [51, 59, 53]:
        cat = 8

    return cat

---

In [28]:
# mid2 = MidiFile('/Users/cooky/Desktop/groove/drummer7/session1/3_rock_86_fill_4-4.mid', clip=True)

for midi in midi_list:
    mid = MidiFile(midi, clip=True)
    on_sec = 0
    event = []

    for msg in mid.tracks[0]:
            note = str(msg).split()
            if len(note) == 5:
                on_sec += msg.dict()['time']

                if note[0] == 'note_on':
                    if [round(on_sec/120), msg.dict()['note']] not in event:
                        event.append([round(on_sec/120), msg.dict()['note']])
    
    beat = pd.DataFrame(0, index = np.arange(9), columns=np.arange( (event[-1][0]//16 + 1) * 16 ))

    for e in event:
        beat.loc[kit_classify(e[1]),e[0]] = 1

    beat.to_csv('./bar_dataset'+midi[27:]+'.csv.gzip', index = False, compression='gzip')

---

In [91]:
imsi = pd.read_csv('bar_dataset/drummer5/session1/10_latin-brazilian-sambareggae_96_beat_4-4.mid.csv.gzip', compression='gzip')

In [92]:
for i in range(imsi.shape[1] // 16 // 2):
    a = random.randrange(0,imsi.shape[1]-32)
    two_bars = imsi.iloc[:,a:a+32]
    two_bars.to_csv(str(i), index=False, compression='gzip')

In [93]:
imsi

,0,1,2,3,4,5,6,7,8,9,...,326,327,328,329,330,331,332,333,334,335
0,0,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
1,0,1,1,1,0,1,0,1,0,1,...,0,1,0,1,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,1,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
imsi.shape[1] // 16 // 2

10

In [90]:
import random

a = random.randrange(0,imsi.shape[1]-32)
imsi.iloc[:,a:a+32]

,3579,3580,3581,3582,3583,3584,3585,3586,3587,3588,...,3601,3602,3603,3604,3605,3606,3607,3608,3609,3610
0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
1,1,1,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0,1,0,1,0,1
